# Prevalence of PRIMIS codelists

In [1]:
import pandas as pd
import numpy as np
import os

suffix = f"_{os.environ.get('OPENSAFELY_BACKEND', 'tpp')}"
os.makedirs(os.path.join("..","safe-outputs"), exist_ok=True)

### Load data

In [2]:
# import first row to get col names
df_head = pd.read_csv(os.path.join("..","output","input.csv"), nrows=1)

# filter out columns not needed
cols_all = df_head.columns
cols_to_use = [c for c in cols_all if c not in ["hashed_organisation", "patient_id"]]

# import full dataset without columns not needed
df = pd.read_csv(os.path.join("..","output","input.csv"), usecols=cols_to_use)

for col in df.columns:
    if col in ["age", "sex"]:
        continue
    # Most columns only contain years or NaN so we can store them as
    # float16s, which saves a lot of memory
    df[col] = df[col].astype("float16")

### Create ageband

In [3]:
agebands = ['16-39', '40-69', '70+']
conditions = [
    (df['age'] >= 16) & (df['age'] < 40),
    (df['age'] >= 40) & (df['age'] < 70),
    (df['age'] >= 70) & (df['age'] < 120)]
choices = agebands
df['ageband'] = np.select(conditions, choices, default=np.nan)

# filter to largest sex groups
df['sex'] = df['sex'].replace(['I','U'], np.nan)


### Summarise data

In [4]:
# list columns of interest 
cols_allyears = [c for c in df.columns if c not in ["age","patient_id"]]
cols_recent = ["preg", "pregdel"]

In [5]:
# filter to valid sexes and agegroups only
df1 = df.copy().loc[(df["sex"].isin(["F","M"])) & (df["ageband"].isin(agebands))]

### Calculate population denominators

In [6]:

out2 = df1.groupby(["ageband", "sex"])[["registered"]].count().rename(columns={"registered":"total_population"}).transpose()

# calculate total population across all ages and sexes
out2["total"] = out2.sum(axis=1)

out2

ageband          16-39       40-69        70+      total
sex                  F     M     F     M    F    M      
total_population  1500  1475  1837  1857  629  632  7930

### Codelist counts

In [7]:
# for codes that are only relevant if recent (pregnancy/delivery), remove any older dates
for c in cols_recent:
    df1.loc[(df1[c]<2020), c] = np.nan

# summarise by age and gender
out = df1[cols_allyears].groupby(["ageband", "sex"]).count().transpose()

# suppress low numbers
out = out.replace([0,1,2,3,4],0)

# calculate total count for each codelist across all ages and sexes
out["total"] = out.sum(axis=1)

# add population denominators
out = pd.concat([out,out2])

display(out.tail())

# export to csv

out_exp = out.copy()
out_exp = 5*((out_exp/5).round(0)).astype(int).replace([0,5], "<8")
out_exp.to_csv(os.path.join("..","safe-outputs",f"code-counts-by-age-and-sex{suffix}.csv"))


ageband          16-39       40-69        70+      total
sex                  F     M     F     M    F    M      
shield             152   140   173   193   60   70   788
smhres             145   118   191   207   61   58   780
spln_cov           147   156   195   171   80   61   810
registered        1500  1475  1837  1857  629  632  7930
total_population  1500  1475  1837  1857  629  632  7930

### Codelist prevalence rates

In [8]:
# calculate rates
for i in out.index.drop("total_population"):
    out.loc[i] = (1000*out.loc[i]/out.loc["total_population"]).round(1)
    
# export to csv    
out.to_csv(os.path.join("..","safe-outputs",f"code-prevalence-by-age-and-sex{suffix}.csv"))

out

ageband            16-39           40-69             70+           total
sex                    F       M       F       M       F       M        
ast                 90.7    97.6   101.8   107.7   108.1    88.6    99.7
astadm              92.7   100.3   100.7    99.6   100.2   113.9    99.9
astrx               91.3    92.2    96.4   110.9    98.6   104.4    98.9
bmi                109.3    96.3   102.3    91.0   109.7    94.9    99.9
bmi_stage           92.0    96.9    99.1   100.2    98.6   121.8    99.4
carehome            99.3   104.4   106.2    95.3   109.7   102.8   102.0
carer              100.7   101.7    95.3   102.3    84.3    98.1    98.5
chd_cov            104.0   103.1    98.5   102.9    85.9    91.8    99.9
ckd15              100.7   103.1    98.0   103.9   101.7    91.8   100.6
ckd35              102.0    98.3   104.5   100.2    95.4   104.4   101.1
ckd_cov            103.3    89.5   108.3    98.5    90.6    85.4    98.4
cld                 98.7   105.1    95.8    95.9    98.6   107.6    99.2
cns_cov             92.7   103.7    89.3   109.3   108.1    91.8    99.0
cov1decl           109.3   100.3   103.4    98.5    92.2    98.1   101.5
cov2decl           102.0   103.7   106.7   115.8   106.5    90.2   106.1
covadm1            112.0   107.1   101.8    98.0   109.7    99.7   104.3
covadm2            129.3    94.2    89.8   100.7   103.3    96.5   102.3
covcontra          108.0    99.0    93.6   100.2   100.2    98.1    99.7
covrx               94.0   115.3    94.2    94.8   111.3    98.1    99.9
diab                90.0   109.8    96.4   103.4    77.9   126.6   100.3
dmres              108.7    98.3    99.1    96.9   109.7    90.2   100.4
domcare            100.7   113.2    95.3    93.2   114.5    82.3    99.6
eth2001             96.7   100.3   100.2   101.8   117.6    96.5   101.0
eth_norecord       111.3   101.0    87.6   103.4   108.1   104.4   101.3
eth_notgiptref     102.0    98.3    94.2    90.5   106.5   123.4    98.9
eth_notstated       92.0    96.3   112.1    91.5    87.4    99.7    97.6
hhld_imdef         108.7    80.7    97.4   100.7    87.4    94.9    96.2
immdx_cov          109.3   105.8    91.5    91.0   100.2   115.5   100.0
immrx               90.7    97.6   104.5    95.9   101.7   120.3    99.6
learndis           104.0    99.0   101.8   104.5    98.6   104.4   102.3
longres            101.3    96.3    98.0   107.7    95.4    93.4   100.0
non_eth2001         98.7   105.1   100.2   103.4   111.3   101.3   102.5
nonshield           89.3   101.7   101.8   106.6    87.4   109.2   100.0
notcarer           110.7   103.7    92.5    96.9    93.8   106.0   100.3
nursehome           84.0    95.6    96.4   111.5   103.3   110.8    99.1
preg                 0.0     4.1     2.7     2.7     0.0     0.0     2.0
pregdel              0.0     0.0     4.4     0.0     0.0     0.0     1.0
resp_cov            97.3    96.9    98.5    98.5   116.1    96.5    99.2
sev_mental         102.7    93.6    99.6    96.9   100.2   112.3    99.5
sev_obesity         88.0    84.7   102.9   108.2   108.1   118.7    99.6
shield             101.3    94.9    94.2   103.9    95.4   110.8    99.4
smhres              96.7    80.0   104.0   111.5    97.0    91.8    98.4
spln_cov            98.0   105.8   106.2    92.1   127.2    96.5   102.1
registered        1000.0  1000.0  1000.0  1000.0  1000.0  1000.0  1000.0
total_population  1500.0  1475.0  1837.0  1857.0   629.0   632.0  7930.0